<a href="https://colab.research.google.com/github/sanagahoi/Data_science_Project/blob/main/Text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import files
import nltk
import string
nltk.download('punkt')
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
from nltk.tokenize import RegexpTokenizer , sent_tokenize
from urllib.request import urlopen
!pip install fake_useragent
from fake_useragent import UserAgent
import urllib.request,sys,time ,requests


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
stopWordsFile =     '/content/drive/MyDrive/Test assignment/StopWords/StopWords_Generic.txt'
positiveWordsFile = '/content/drive/MyDrive/Test assignment/MasterDictionary/positive-words.txt'
nagitiveWordsFile = '/content/drive/MyDrive/Test assignment/MasterDictionary/negative-words.txt'

In [ ]:
input = pd.read_excel('/content/drive/MyDrive/Test assignment/Input.xlsx')
input.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [ ]:
def get_article_names(urls):
  titles = []
  for i in range (len(urls)):
    title = urls[i]
    title_clean = title[title.index( "m/" ) + 2 :-1]. replace('-' , ' ')
    titles.append(title_clean)
  return titles

urls =input["URL"]
urlsTitleDF = get_article_names(urls)
urlsTitleDF

['rising it cities and its impact on the economy environment infrastructure and city life by the year 2040 2',
 'rising it cities and their impact on the economy environment infrastructure and city life in future',
 'internet demands evolution communication impact and 2035s alternative pathways',
 'rise of cybercrime and its effect in upcoming future',
 'ott platform and its impact on the entertainment industry in future',
 'the rise of the ott platform and its impact on the entertainment industry by 2040',
 'rise of cyber crime and its effects',
 'rise of internet demand and its impact on communications and alternatives by the year 2035 2',
 'rise of cybercrime and its effect by the year 2040 2',
 'rise of cybercrime and its effect by the year 2040',
 'rise of internet demand and its impact on communications and alternatives by the year 2035',
 'rise of telemedicine and its impact on livelihood by 2040 3 2',
 'rise of e health and its impact on humans by the year 2030',
 'rise of e he

In [ ]:
def extract_data(url):
  try:
    page=requests.get(url , headers={"User-Agent": "XY"})
  except:
    print("can't get response")

  try:
    soup = BeautifulSoup(page.text , 'html.parser')
  except:
    print("Can't get page ")

  text = ''
  try:
    #get title
    title = soup.find("h1",attrs = { 'class' : 'entry-title'}).get_text()
    #get article text
    text = soup.find(attrs = { 'class' : 'td-post-content'}).get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
  except:
    print("Can't get data for {}".format(url))

  return text


In [ ]:
# Loading positive words
with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')


# Loading negative words
with open(nagitiveWordsFile ,'r' ,  encoding="ISO-8859-1") as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

#Loading stop words dictionary for removing stop words

with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')
stopWordList[-1:] = []

display( positiveWordList[:6]  , negativeWordList[:6] , stopWordList[:6])

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable']

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably']

['about', 'above', 'after', 'again', 'all', 'am']

In [ ]:
#tokenizeing module and filtering tokens using stop words list, removing punctuations
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

def positive_score (text):
  posword=0
  tokenphrase = tokenizer(text)
  for word in tokenphrase :
    if word in positiveWordList:
      posword+=1

    retpos = posword
    return retpos

def negative_score (text):
  negword=0
  tokenphrase = tokenizer(text)
  for word in tokenphrase :
    if word in negativeWordList :
      negword +=1

    retneg = negword
    return retneg

def polarity_score (positive_score , negative_score) :
  return (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)

def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

def averageSentenceLength (text):
  Wordcount = len(tokenizer (text))
  SentenceCount = len (sent_tokenize(text))
  Average_Sentence_Length =0
  if SentenceCount > 0 : Average_Sentence_Length = Wordcount / SentenceCount

  avg = Average_Sentence_Length

  return round(avg)

# Counting complex words
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0

    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0

    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)

    return complex_word_percentage

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex


In [ ]:
URLS = input ["URL"]
URLS

0     https://insights.blackcoffer.com/rising-it-cit...
1     https://insights.blackcoffer.com/rising-it-cit...
2     https://insights.blackcoffer.com/internet-dema...
3     https://insights.blackcoffer.com/rise-of-cyber...
4     https://insights.blackcoffer.com/ott-platform-...
                            ...                        
95    https://insights.blackcoffer.com/what-is-the-r...
96    https://insights.blackcoffer.com/impact-of-cov...
97    https://insights.blackcoffer.com/contribution-...
98    https://insights.blackcoffer.com/how-covid-19-...
99    https://insights.blackcoffer.com/how-will-covi...
Name: URL, Length: 100, dtype: object

In [ ]:
corps =[]

for url in URLS:
  data = extract_data(url)
  corps.append(data)


Can't get data for https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/
Can't get data for https://insights.blackcoffer.com/how-advertisement-increase-your-market-value/
Can't get data for https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/
Can't get data for https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Can't get data for https://insights.blackcoffer.com/future-of-work-how-ai-has-entered-the-workplace/
Can't get data for https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Can't get data for https://insights.blackcoffer.com/human-rights-outlook/
Can't get data for https://insights.blackcoffer.com/how-voice-search-makes-your-business-a-successful-business/
Can't get data for https://insights.blackcoffer.com/estimating-the-impact-of-covid-19-on-the-world-of-work-3/
Can't get data for https://insights.blackcoffer.com/how-covid-19-is-impactin

In [ ]:
#df = pd.DataFrame(corps,urlsTitleDF)
df = pd.DataFrame({'title':urlsTitleDF,'corps': corps})

df["total word count"] = df["corps"].apply(total_word_count)
df["percentage_complex_word"] = df["corps"].apply (percentage_complex_word)
df["complex_word_count"] = df["corps"].apply (complex_word_count)
df["AverageSentenceLength"] = df["corps"].apply (averageSentenceLength)
df["positive_score"] = df["corps"].apply (positive_score)
df["negative_score"] = df["corps"].apply (negative_score)
df["polarity_score"] = np.vectorize(polarity_score)(df['positive_score'],df['negative_score'])

df

,title,corps,total word count,percentage_complex_word,complex_word_count,AverageSentenceLength,positive_score,negative_score,polarity_score
0,rising it cities and its impact on the economy...,We have seen a huge development and dependence...,726,0.282369,205,9,0.0,0.0,0.0
1,rising it cities and their impact on the econo...,"Throughout history, from the industrial revolu...",946,0.402748,381,12,0.0,0.0,0.0
2,internet demands evolution communication impac...,Introduction\nIn the span of just a few decade...,727,0.462173,336,13,0.0,0.0,0.0
3,rise of cybercrime and its effect in upcoming ...,"The way we live, work, and communicate has unq...",716,0.470670,337,14,0.0,0.0,0.0
4,ott platform and its impact on the entertainme...,The year 2040 is poised to witness a continued...,458,0.342795,157,12,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
95,what is the repercussion of the environment du...,"Epidemics, in general, have both direct and in...",708,0.347458,246,14,0.0,0.0,0.0
96,impact of covid 19 pandemic on office space an...,COVID 19 has bought the world to its knees. Wi...,603,0.283582,171,16,0.0,0.0,0.0
97,contribution of handicrafts visual arts litera...,Handicrafts is an art of making crafts by hand...,271,0.409594,111,23,0.0,0.0,0.0
98,how covid 19 is impacting payment preferences,,0,0.000000,0,0,NaN,NaN,NaN


In [ ]:
final_df = df.drop("corps" , 1)
final_df

<ipython-input-12-f2c2632df826>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  final_df = df.drop("corps" , 1)


,title,total word count,percentage_complex_word,complex_word_count,AverageSentenceLength,positive_score,negative_score,polarity_score
0,rising it cities and its impact on the economy...,726,0.282369,205,9,0.0,0.0,0.0
1,rising it cities and their impact on the econo...,946,0.402748,381,12,0.0,0.0,0.0
2,internet demands evolution communication impac...,727,0.462173,336,13,0.0,0.0,0.0
3,rise of cybercrime and its effect in upcoming ...,716,0.470670,337,14,0.0,0.0,0.0
4,ott platform and its impact on the entertainme...,458,0.342795,157,12,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
95,what is the repercussion of the environment du...,708,0.347458,246,14,0.0,0.0,0.0
96,impact of covid 19 pandemic on office space an...,603,0.283582,171,16,0.0,0.0,0.0
97,contribution of handicrafts visual arts litera...,271,0.409594,111,23,0.0,0.0,0.0
98,how covid 19 is impacting payment preferences,0,0.000000,0,0,NaN,NaN,NaN


In [ ]:
final_df.to_excel('Output Data Structure.xlsx', encoding='utf-8', index=False)

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [ ]:
from operator import index
final_df.head()

TypeError: ignored